# Using loops in Julia

![Image of Fruitfly](https://upload.wikimedia.org/wikipedia/commons/c/cb/Drosophila_pseudoobscura-Male.png)

The Bicoid protein is expressed in the fruit fly embryo (Drosophila melanogaster) with a concentration gradient that is maximal in the anterior pole (head) and decreases toward the posterior one (tail). This gradient causes the differential expression of other genes that will eventually lead to the formation of the typical segmented body of the fly. This is one of the most studied case of pattern formation or morphogensis in biology.

Here we will look at the concentration of Bicoid along the anteroposterior axis, and try to understand how such a gradient can be established using a simple model. This example illustrates how one can easily implement a simple mathematical model in Julia directly using for loops, without having to worry about the peformance issues that languages that rely on vectorization have. We will also use in-place modification of inputs and disable bounds checking.

## Gadfly
Install `Gadfly` (for plotting) if you don't already have it.

In [ ]:
Pkg.add("Gadfly")

Import `Gadfly`.

In [ ]:
using Gadfly

## Data

In [ ]:
include("data.jl")
plot(x=x, y=bcd, Guide.xlabel("A-P axis position"), Guide.ylabel("Bcd concentration"))

The gradient looks like an exponential decay, let's make a model for it.

We will assume that the protein is produced at the anterior pole and that it diffuses and is degraded in the rest of the egg.

We first we need to compute the second derivative (or Laplacian Δ) of the concentration, so we can simulate diffusion. We use a simple finite differences scheme. The function is annotated with a conventional ! to indicate that it modifies the array d.

Note: Julia provides autocompletion for mathematical symbols, e.g. type "\Delta" and press tab.

In [ ]:
function Δ!(f,δx,d)  # Pass the output vector as argument to avoid allocations
    Nx = length(f) 
    δx = δx^2
    @inbounds for i=2:Nx-1  # Disable bounds checking
        d[i] = (f[i+1] -2*f[i] + f[i-1])/δx
    end
    d[1] = (f[2] - f[1])/δx  # Boundary condition f[-1] = f[1]
    d[Nx] = (f[Nx-1] -f[Nx])/δx  # f[Nx] = f[Nx+1]
end

We can then simulate the following degration-diffusion equation, with a localized source, using the Euler method: f'(x,t) = αδ(x-x0) -γf(x,t) + DΔf(x,t)

In [ ]:
function simulate(f0,Nt,δt,δx,α,γ,D)
    
    Nx = length(f0)
    f = f0        # Keep the last time point (near steady state)
    d = zeros(Nx) # Vector to store the Laplacian
    
    for t=2:Nt
        Δ!(f,δx,d)
        f[1] += δt*α/δx # Production
        @inbounds for i=1:Nx 
            f[i] += δt*( D*d[i] -γ*f[i] ) # Pegradation and diffusion
        end
    end
    f
end

Finally we can simulate the model for a given set of parameters and check if it matches the data. One can play with the parameters to improve the fit. 

Because Julia's loops are fast (the simulation runs under 0.2 second, with Nt*Nx = 4e6) it would be easy to write an error (likelihood) function and to fit the model to the data, or to extend the model to include mRNA, or other genes.

In [ ]:
Nx = 200
fx = collect(linspace(minimum(x),maximum(x),Nx))
δt = 0.01
Nt = round(200/δt) #simulate for 200 minutes
δx = fx[2]-fx[1]

#parameters
D = 40
α = 200
γ = 1/65

f0 = zeros(Nx)
f = simulate(f0,Nt,δt,δx,α,γ,D);
    
plot(
    layer( x=x, y=bcd,Geom.point ),
    layer( x=fx, y=f, Geom.line, Theme(default_color=colorant"black"))
)

Because we were able to do in-place modification of the inputs, we use almost no memory allocations. (It's fast!)

In [ ]:
@time f = simulate(f0,Nt,δt,δx,α,γ,D);